In [137]:
import pandas as pd
import numpy as np
import pint
import pint_pandas
import iam_units
import dvc_pandas
from pint_pandas.pint_array import PintArray
from typing import Dict
#import nodes.context

FORECAST_COLUMN = 'Forecast'
FORECAST_x = 'Forecast_x'
FORECAST_y = 'Forecast_y'
VALUE_COLUMN = 'Value'
VALUE_x = 'Value_x'
VALUE_y = 'Value_y'

unit_registry = pint.UnitRegistry(
    preprocessors=[
        lambda s: s.replace('%', ' percent '),
    ],
    on_redefinition='raise'
)

# By default, kt is knots, but here kilotonne is the most common
# usage.
del unit_registry._units['kt']
unit_registry.define('kt = kilotonne')
# Mega-kilometers is often used for mileage
unit_registry.define('Mkm = gigameters')
unit_registry.define(pint.unit.UnitDefinition(
    'percent', '%', (), pint.converters.ScaleConverter(0.01)
))
unit_registry.define('ingestion_p1 = mg/d')
unit_registry.define('disability_adjusted_life_year = [disease_burden] = DALY')
unit_registry.define('Lden = []')
unit_registry.define('microbe = [] = microbes')
unit_registry.define('parts_per_million = mg / kg = ppm')
unit_registry.define('personyear = person * year')
unit_registry.define('euro = [] = EUR')
unit_registry.define('toxic_equivalency_factor = [tef] = TEF')
unit_registry.define('MET = [metabolic_equivalent]')
unit_registry.define('METh = MET * h')

unit_registry.default_format = '~P'
pint.set_application_registry(unit_registry)
pint_pandas.PintType.ureg = unit_registry  # type: ignore

In [157]:
df = pd.DataFrame({
    'A': pd.Series([2,4]),
    'Value': pd.Series([1.,3.], dtype='pint[mg/d]'),
    'erf': pd.Series(['RR', 'Step'])
}).set_index(['A'])

#df.loc[df.erf=='Step'].reset_index().Value[0]
newind = df.index.names + ['erf']
df.reset_index().set_index(newind)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pint_pandas/pint_array.py:648: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  return np.array(qtys, dtype="object", copy=copy)


,,Value
A,erf,
2,RR,1.0
4,Step,3.0


In [147]:
#        exposure[VALUE_COLUMN] = exposure[VALUE_COLUMN].pint.to('mg/kg/d', 'exposure_generic')
def check_erf_units(param):
    out = unit_registry(param)
    is_erf_compatible = df6.Value.pint.units.is_compatible_with(out)
    if not is_erf_compatible:
        print(df6.Value)
#        df6.Value = df6.Value.pint.to('kg/a')
        out = out.to('kLden')
    return out

check_erf_units('Lden')

0    1.0
1    3.0
Name: Value, dtype: pint[mg/d]


0.001 <Unit('kiloLden')>

In [150]:
powers = {
    'p1': 'mg/kg/d',
    'p0': 'dimensionless',
    'm1': 'kg d / mg',
    'm2': '(kg d / mg)**2',
    'm3': '(kg d / mg)**3'
}
powers['m1']

'kg d / mg'